### 作業目的: 使用樹型模型進行文章分類

本次作業主利用[Amazon Review data中的All Beauty](https://nijianmo.github.io/amazon/index.html)來進行review評價分類(文章分類)

資料中將review分為1,2,3,4,5分，而在這份作業，我們將評論改分為差評價、普通評價、優良評價(1,2-->1差評、3-->2普通評價、4,5-->3優良評價)

### 載入套件

In [1]:
import json
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

### 資料前處理
文本資料較為龐大，這裡我們取前10000筆資料來進行作業練習

In [2]:
#load json data
all_reviews = []
with open('All_Beauty.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        all_reviews.append(json.loads(line))
        
all_reviews[0]

{'overall': 1.0,
 'verified': True,
 'reviewTime': '02 19, 2015',
 'reviewerID': 'A1V6B6TNIC10QE',
 'asin': '0143026860',
 'reviewerName': 'theodore j bigham',
 'reviewText': 'great',
 'summary': 'One Star',
 'unixReviewTime': 1424304000}

In [3]:
#parse label(overall) and corpus(reviewText)
corpus = []
labels = []

for review in all_reviews[:10000]:
    try:
        corpus.append(review['reviewText'])
        rating = review['overall']
        #transform labels: 1,2 --> 1 and 3 --> 2 and 4,5 --> 3
        if rating <= 1:
            labels.append(1)
        elif rating == 3:
            labels.append(2)
        else:
            labels.append(3)
    except:
        pass

In [4]:
#preprocessing data
#remove email address, punctuations, and change line symbol(\n)
import string
for i,txt in enumerate(corpus):
#     txt = re.compile('[^\@]\@[^\@]')
    txt = re.sub("["+string.punctuation+"]","",txt)
    txt = re.sub(r"\n"," ",txt)
    corpus[i] = txt

In [5]:
#split corpus and label into train and test
x_train, x_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.3, random_state=42)

len(x_train), len(x_test), len(y_train), len(y_test)

(6996, 2999, 6996, 2999)

In [6]:
#change corpus into vector
#you can use tfidf or BoW here

tf = TfidfVectorizer(max_features=2000)
tf.fit(x_train)

#transform training and testing corpus into vector form
x_train = tf.transform(x_train)
x_test = tf.transform(x_test)

### 訓練與預測

In [7]:
#build classification model (decision tree, random forest, or adaboost)
#start training

clf = DecisionTreeClassifier(max_depth=6)
clf.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=6)

In [8]:
#start inference
y_pred = clf.predict(x_test)

In [9]:
from sklearn import metrics
#calculate accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
print(accuracy)

0.9226408802934312


In [10]:
#calculate confusion matrix, precision, recall, and f1-score
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.60      0.18      0.28       136
           2       0.00      0.00      0.00       110
           3       0.93      1.00      0.96      2753

    accuracy                           0.92      2999
   macro avg       0.51      0.39      0.41      2999
weighted avg       0.88      0.92      0.89      2999

[[  25    1  110]
 [   7    0  103]
 [  10    1 2742]]


由上述資訊可以發現, 模型在好評的準確度高(precision, recall都高), 而在差評的部分表現較不理想, 在普通評價的部分大部分跟差評搞混,
同學可以試著學習到的各種方法來提升模型的表現